# 01_data_ingestion

In [1]:
import os

In [2]:
%pwd

'd:\\Niranjan\\Personal Docs\\Data Science\\Projects\\Project\\Chicken-Disease-Classification-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Niranjan\\Personal Docs\\Data Science\\Projects\\Project\\Chicken-Disease-Classification-Project'

## Update the entity:

In [7]:
from dataclasses import dataclass #Define the custom data type of return of the functions
from pathlib import Path

@dataclass(frozen=True) #Setting (frozen=True) makes the instance immutable
class DataIngestionConfig:
    # dir: Data type => this will come from config.yaml
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path 

## Update Configuration Manager in src/config

In [5]:
from CNN_Classifier.constants import *
from CNN_Classifier.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH, # From constant.py
                 params_filepath = PARAMS_FILE_PATH): # From constant.py 
        
        self.config = read_yaml(config_filepath) # assigns dict from config/config.yaml
        self.params = read_yaml(params_filepath) # assigns dict from param.yaml
        
        create_directories([self.config.artifacts_root]) #Creating artifact folder in root directory
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir]) #creating data_ingestion folder within a artifact directory
        
        # define return type of the function and store the values in a variable
        data_ingestion_config = DataIngestionConfig(
            root_dir= config.root_dir,
            source_URL= config.source_URL,
            local_data_file= config.local_data_file,
            unzip_dir= config.unzip_dir
        )
        
        return data_ingestion_config


## Update the Components

In [10]:
import os
import urllib.request as request
import zipfile
from CNN_Classifier import logger
from CNN_Classifier.utils.common import get_size

In [22]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            # Extracting the zip file from the link and storing it in the given location
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename= self.config.local_data_file
            )        
            logger.info(f"{filename} download ! with following info: \n{headers}")
            
        else:
            logger.info(f"File already exists of size : {get_size(Path(self.config.local_data_file))}")
            
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        The extracted zip file is saved to the give folder
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(name=unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

## Pipeline

In [23]:
try :
    """
    config.yaml file is created to define paths for various files extracted.
    Such comments can't be written in yaml file.
    """
    config = ConfigurationManager()
    # Get the locations of all the folder
    data_ingestion_config = config.get_data_ingestion_config()
    # Instantiate the DataIngestion class with attribute "data_ingestion_config"
    data_ingestion = DataIngestion(config=data_ingestion_config)
    # Download the zip folder
    data_ingestion.download_file()
    # Extract the zip files
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e

[2024-07-07 13:42:05,042: INFO: common: YAML file :config\config.yaml loaded successfully]
[2024-07-07 13:42:05,052: INFO: common: YAML file :params.yaml loaded successfully]


[2024-07-07 13:42:05,052: INFO: common: created directory at: artifacts]
[2024-07-07 13:42:05,070: INFO: common: created directory at: artifacts/data_ingestion]
[2024-07-07 13:42:05,077: INFO: 803794734: File already exists of size : ~ 11345 KB]
